# HTML解析入门及准备URL生成连续技
![for humans](https://requests-html.kennethreitz.org/_static/requests-html-logo.png#thumbnail)

*  本周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  上周主要内容：HTML解析（parse HTML）及Xpath实践
*  20春_Web数据挖掘_week03
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* HTML解析（parse HTML）: requests-html  丶
* Xpath实践
* m.liepin.com 取工作牛肉

-----
## 本周内容及学习目标

本周内容聚焦在

<mark> 如何从一页开始有系统的找更多页的内容 </mark>

为此，我们需要学习

1. 拆解带有参数的URL，并再从query取出参数
   a. URL拆解: 使用 urllib.parse 解析 出query
   b. query拆解:  取出参数 成python字典
2. 有基底URL，加上参数字典，请求新网页连续技

我们除了继续学习解决上一周已开始面对的以下挑战：
![Xpath Axis](http://krum.rz.uni-mannheim.de/inet-2005/images/xpath-axis.gif)

### 旧目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)

### 新目标
这一周，学生将实践
* 猎聘PC版 liepin.com 取工作URL参数的牛肉
* 如何生成一连串新URL以进一步爬取数据




In [1]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [6]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

# 0. 上周加分作业解答

In [7]:
# C-1   单一页面
url = "https://m.liepin.com/zhaopin/?keyword=python"
session = HTMLSession()
r = session.get( url )

# C-5
# 难: '公司URL', '时间', '经验'

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//div[@class="job-card-wrap"]//div[@class="job-card"]')

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        '经验':      './/ul/li[time]/text()'
    },
    'text_content': {
        '职称':    './/ul/li/a[contains(@class,"job-name")]/span[@class="name-text"]', 
        '薪水':    './/ul/li/a[contains(@class,"job-name")]/following-sibling::span', 
        '公司地点':'.//ul/li/time/following-sibling::a',
        '公司名称': './/ul/li/a[contains(@class,"company-name")]', 
        '时间':    './/ul/li/time', 
    },
    'href': {
        '链结':    './/ul/li/a[contains(@class,"job-name")]', 
        '公司URL': './/ul/li/a[contains(@class,"company-name")]', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

print ([len(v) for k,v in 数据字典.items()])  # 檢查

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week02_liepin.xlsx", sheet_name="搜查结果")
数据 

[3, 3, 3, 3, 3, 3, 3, 3]


,职称,薪水,公司地点,公司名称,时间,经验,链结,公司URL
0,嵌入式软件开发工程师,12-18k·12薪,肇庆,广东国腾量子科技有限公司,2020-04-01,6年以上 本科及以上,https://m.liepin.com/job/1925309167.shtml,https://m.liepin.com/company/9831379/
1,中高级数据分析师(J10421),10-20k·12薪,肇庆,岩心科技,一个月前,3年以上 本科及以上,https://m.liepin.com/job/1922670795.shtml,https://m.liepin.com/company/8640905/
2,软件开发工程师,7-12k·12薪,肇庆,东莞怡合达自动化股份有限公司,一个月前,3年以上 本科及以上,https://m.liepin.com/job/1916098579.shtml,https://m.liepin.com/company/9429508/



-----

# 本周目标
* [猎聘PC版](https://www.liepin.com/zhaopin/)
* 上方导航有  公司行业 城市 薪资 的分页选单
* 请练习xpath抽出数据

## Xpath解析HTML

In [8]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=python"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,大专及以上,5年以上,面议,2020年04月10日,店长（上海国金，小众香水品牌）,北京,北京恒城实业,https://www.liepin.com/job/1927361185.shtml,https://www.liepin.com/company/1837423/
1,本科及以上,3年以上,8-10k·12薪,2020年04月10日,系统管理员,无锡,无锡八佰伴,https://www.liepin.com/job/1927361121.shtml,https://www.liepin.com/company/1200071/
2,学历不限,经验不限,8-12k·12薪,2020年04月10日,KOL运营,,北京和风清穆影视文化传播有限公司,https://www.liepin.com/job/1927360683.shtml,https://www.liepin.com/company/9546719/
3,大专及以上,1年以上,4-5k·12薪,2020年04月10日,维修技工（水电工）,云浮,翔顺集团,https://www.liepin.com/job/1927360581.shtml,https://www.liepin.com/company/10139521/
4,大专及以上,经验不限,3-4k·12薪,2020年04月10日,监控文员,云浮,翔顺集团,https://www.liepin.com/job/1927360441.shtml,https://www.liepin.com/company/10139521/
5,统招本科,8年以上,15-20k·13薪,2020年04月10日,人力资源经理（物业）,,北京美瑞泰富置业有限公司,https://www.liepin.com/job/1927359591.shtml,https://www.liepin.com/company/8162551/
6,大专及以上,8年以上,10-15k·16薪,2020年04月10日,深圳4S店人力行政经理,,北京惠通陆华汽车服务有限公司,https://www.liepin.com/job/1927359003.shtml,https://www.liepin.com/company/7887986/
7,本科及以上,5年以上,20-30k·12薪,2020年04月10日,产业园区招商总监（农业、物流）,南京,太平洋建设,https://www.liepin.com/job/1927358699.shtml,https://www.liepin.com/company/6593372/
8,大专及以上,2年以上,4-5k·12薪,2020年04月10日,置业顾问（异地招聘）,常州-新北区,东渡国际,https://www.liepin.com/job/1927358493.shtml,https://www.liepin.com/company/7919318/
9,本科及以上,7年以上,20-25k·13薪,2020年04月10日,法务经理,东莞-莞城区,保利达中国置地,https://www.liepin.com/job/1927358431.shtml,https://www.liepin.com/company/8042723/


## 使用urllib3 解析 url 
上面的url应该触动不同的页面查询，但能不能轻松无误的拆分url并进行比较？

### urllib模块功能介绍
* urlparse 
返回的6个部分，分别是：scheme(机制)丶netloc(网络位置)丶path(路径)丶params(路径段参数)丶query(查询)丶fragment(片段)。
* parse_qs
返回query(查询)多个部分

In [9]:
# B-1 使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 公司数据选择器链结.values()]

NameError: name '公司数据选择器链结' is not defined

In [10]:
# A-2 扩张 公司 ?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
               
公司数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
公司数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=478330d5852f8fd7&flushckid=1&fromSearchBtn=2&keyword=python&compTag=155&ckid=478330d5852f8fd7&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=f29ca0775961868d465c0887829f1a79&d_curPage=0&d_pageSize=40&d_headId=f29ca0775961868d465c0887829f1a79',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=478330d5852f8fd7&flushckid=1&fromSearchBtn=2&keyword=python&compTag=182&ckid=478330d5852f8fd7&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=f29ca0775961868d465c0887829f1a79&d_curPage=0&d_pageSize=40&d_headId=f29ca0775961868d465c0887829f1a79',
 '制造业500强': '/zhaopin/?init=-1&headckid=478330d5852f8fd7&flushckid=1&fromSearchBtn=2&keyword=python&compTag=186&ckid=478330d5852f8fd7&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=f29ca0775961868d465c0887829f1a79&d_curPage=0&d_pageSize=40&d_headId=f29ca0775961868d465c0887829f1a79',
 'AI创新成长50强 ': '/zhaopin/?init=-1&

In [11]:
# B-2 使用 pd.DataFrame进行 unuinque()相异值计量比对 
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes
scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=478330d5852f8fd7&flushckid=1&...,


In [12]:
# B-3 针对query 再解析之 
#df_qs = pd.DataFrame([ parse_qs(x) for x in df['query'] ])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]

init             1
headckid         1
flushckid        1
fromSearchBtn    1
keyword          1
compTag          6
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,keyword,compTag
0,python,155
1,python,182
2,python,186
3,python,189
4,python,130
5,python,156


### 小结
* comTag 是不同的公司选择器, 数值不样, 对映到不同类型的公司
* keyword 是搜查关键字

In [13]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)


{'init': ['-1'], 'headckid': ['478330d5852f8fd7'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['python'], 'compTag': ['155'], 'ckid': ['478330d5852f8fd7'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f29ca0775961868d465c0887829f1a79'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f29ca0775961868d465c0887829f1a79']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [14]:
# B-5 建构 参数模板  
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return (参数)

参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
print(参数_compTag_用户体验)

{'中国500强': {'init': ['-1'], 'headckid': ['478330d5852f8fd7'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['478330d5852f8fd7'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f29ca0775961868d465c0887829f1a79'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f29ca0775961868d465c0887829f1a79']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['478330d5852f8fd7'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['478330d5852f8fd7'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['f29ca0775961868d465c0887829f1a79'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['f29ca0775961868d465c0887829f1a79']}, '制造业500强': {'init': ['-1'], 'headckid': ['478330d5852f8fd7'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['478330d5852f8fd7'], '

## requests 生成

In [15]:
# C-1   多个页面准备测试1 中国500强
# 尝试解析2018年互联网300强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['2018互联网300强']
r = session.get( url, params = payload)
r.url

'https://www.liepin.com/zhaopin/?init=-1&headckid=478330d5852f8fd7&flushckid=1&fromSearchBtn=2&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&compTag=182&ckid=478330d5852f8fd7&siTag=1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=f29ca0775961868d465c0887829f1a79&d_curPage=0&d_pageSize=40&d_headId=f29ca0775961868d465c0887829f1a79'

In [16]:
# C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)




In [17]:
# C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL,热门公司类型
0,本科及以上,5年以上,20-30k·12薪,2020年04月10日,产业园区招商总监（农业、物流）,南京,太平洋建设,https://www.liepin.com/job/1927358699.shtml,https://www.liepin.com/company/6593372/,中国500强
1,统招本科,5年以上,面议,2020年04月10日,集团总部招采总监,深圳-南山区,禹洲地产股份有限公司,https://www.liepin.com/job/1927352801.shtml,https://www.liepin.com/company/2245242/,中国500强
2,统招本科,5年以上,13-18k·14薪,2020年04月10日,机场场道技术员（施工技术标书）,,远东控股集团,https://www.liepin.com/job/1927286259.shtml,https://www.liepin.com/company/2892805/,中国500强
3,本科及以上,5年以上,12-18k·14薪,2020年04月10日,绩效薪酬经理（总部）,成都,通威股份,https://www.liepin.com/job/1926610481.shtml,https://www.liepin.com/company/682357/,中国500强
4,统招本科,5年以上,13-18k·14薪,2020年04月10日,场道职员或副经理,,远东控股集团,https://www.liepin.com/job/1919851771.shtml,https://www.liepin.com/company/2892805/,中国500强
...,...,...,...,...,...,...,...,...,...,...
35,统招本科,5年以上,面议,2020年04月02日,教学产品经理,武汉-武昌区,新东方教育科技集团有限公司,https://www.liepin.com/job/1926694451.shtml,https://www.liepin.com/company/2485876/,上市公司
36,统招本科,2年以上,10-15k·12薪,2020年04月02日,数据库开发工程师,上海-张江,空中信使信息技术北京,https://www.liepin.com/job/1926600865.shtml,https://www.liepin.com/company/7880101/,上市公司
37,本科及以上,5年以上,30-50k·12薪,2020年03月31日,运营总监/经理,深圳-西丽,禹洲地产股份有限公司,https://www.liepin.com/job/1927104335.shtml,https://www.liepin.com/company/2245242/,上市公司
38,大专及以上,2年以上,10-15k·12薪,2020年03月28日,装修工程师,杭州,宋城集团,https://www.liepin.com/job/1927040819.shtml,https://www.liepin.com/company/1043007/,上市公司


In [18]:
# C-4   输出
df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

In [19]:
# C-5 Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司地点','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         5
经验         10
薪水         70
时间         24
职称        180
公司地点       68
公司名称       70
链结        190
公司URL      70
热门公司类型      6
dtype: int64


职称
公司地点    edu      
杭州      学历不限   44
北京      统招本科   15
        本科及以上  11
        统招本科   10
        本科及以上   8
...            ..
南昌      大专及以上   1
上海-中山公园 统招本科    1
武汉-江夏区  本科及以上   1
南宁      大专及以上   1
上海-杨浦区  本科及以上   1

[94 rows x 1 columns]

In [20]:
df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

职称
公司名称           edu      
阿里巴巴           学历不限   45
小米             统招本科   12
明略科技集团         统招本科   11
华为             本科及以上  10
               统招本科    8
...                   ..
欧菲光            本科及以上   1
博世             统招本科    1
深圳市优必选科技股份有限公司 统招本科    1
深圳市农产品集团股份有限公司 本科及以上   1
深圳越海全球供应链有限公司  学历不限    1

[93 rows x 1 columns]

In [21]:
# 输出城市
df_all.groupby(['公司地点','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

职称
公司地点    edu      
杭州      学历不限   44
北京      统招本科   15
        本科及以上  11
        统招本科   10
        本科及以上   8
...            ..
南昌      大专及以上   1
上海-中山公园 统招本科    1
武汉-江夏区  本科及以上   1
南宁      大专及以上   1
上海-杨浦区  本科及以上   1

[94 rows x 1 columns]

In [22]:
#输出薪资
df_all.groupby(['薪水','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

职称
薪水         edu      
面议         学历不限   48
           统招本科   37
           本科及以上  20
           大专及以上   7
8-12k·13薪  本科及以上   4
...               ..
20-30k·15薪 统招本科    1
20-30k·12薪 本科及以上   1
18-30k·12薪 统招本科    1
16-30k·14薪 统招本科    1
25-35k·12薪 统招本科    1

[92 rows x 1 columns]

# 本周练习

一样反向工程解析:

## 上方界面的params参数
* 公司：v
* 行业：换成python
* 城市：v
* 薪资：v
## 下方界面的params参数
* 跳转到 N 页确定 ?
## 换  
* keyword换成python
